
# Introduction to Gradient Descent method (most important lesson in machine learning)

* Basic probelm in machine learning is to find best model for a certain situation
* BEST will mean something like **minimizes the error between model and data**. 
$$\sum\limits_{i=1}^n e_{i}^2 = \sum\limits_{i=1}^n (y_{i}^{actual} - y_{i}^{model})^2 = \sum\limits_{i=1}^n (y_{i}^{actual} - (\beta_{0} + \beta_{1}x_{i}))^2$$
* In other words, it will represent the solution to some sort of optimization problem.
* Finding the optimum solution requires a technique called gradient descent
* We will explore the use of gradient descent to determine our parameters for linear regression

<tr>
    <td> <img src="img/least-squares2.svg" alt="Drawing" style="width: 300px;"/> </td>
    <td> <img src="img/best_fitline.png" alt="Drawing" style="width: 500px;"/> </td>
    </tr>

# Table of Content
<ul>
    <li> <a href="#gripsearch"> Grid Search </a> </li>
    <li> <a href="#gradientdescent"> Gradient Descent  </a> </li>
    <li> <a href="#multivariate"> Multivariate linear regression  </a> </li>
</ul>

In [ ]:
%matplotlib inline 
import sys
import re
import numpy as np
from scipy import stats, polyfit
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter("ignore")

<a id='gridsearch'></a>
# Grid Search

## One dimensional example

let us say we have following function 

$$f(x) = 2x^{2} + 2x$$

Now, I want to find the value of x where this function is minimum

$$\frac{\partial }{\partial x} f(x) = 4x + 2 = 0$$

which gives $$x = -0.5$$

So analytical solution gives $x=-0.5$, now we will try to find this value numerically


In [ ]:
def func_onedim(x):
    return 2*x*x + 2*x

In [ ]:
x = np.linspace(-5,5,4000)
y = func_onedim(x)

In [ ]:
plt.plot(x,y,linestyle='', marker='o')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('variation of f(x) as a function of x')
plt.show()

In [ ]:
min_index = np.argmin(y)
print("Minimum value of f(x) lies at x = {:.3f}".format(x[min_index]))

Lets us find the minimum value of function using standard scipy package

In [ ]:
from scipy.optimize import minimize
result = minimize(func_onedim, x0=-1)
print("Minimum value of f(x) lies at x = {:.3f}".format(result.x[0]))

We’re going to create a linear regression that follows the following relation:

$$y=\beta_{0}+\beta_{1}x$$
where:
$$\beta_{0}=5$$
$$\beta_{1}=2$$

Let’s start by creating some mock data and we’ll graph up this data.

## Lets us apply the grid search method for our linear regression problem

In [ ]:
beta_0 = 5
beta_1 = 2
x= np.random.randint(10,100, 200)
y = beta_0 + beta_1*x

# Add some random error

y = y + np.random.normal(0,10, len(x))

In [ ]:
plt.plot(x,y, linestyle='', marker='o')
plt.xlabel('x')
plt.ylabel('y')
plt.title('x vs y regression')
plt.show()

The standard notation is:

$$y^{model} = \beta_{0} + \beta_{1}x$$

where $\beta_{0}$, $\beta_{1}$ are regression coefficients we need to determine

Lets us define the error as;

$$e_{i} = y_{i}^{actual} - y_{i}^{model}$$ 

From this error definition, we can find the expression for square of error terms for all data points as:

$$cost function \ (Mean \ Squared \ error) = \frac{1}{2n}\sum\limits_{i=1}^n e_{i}^2 = \frac{1}{2n} \sum\limits_{i=1}^n (y_{i}^{actual} - y_{i}^{model})^2 = \frac{1}{2n} \sum\limits_{i=1}^n (y_{i}^{actual} - (\beta_{0} + \beta_{1}x_{i}))^2$$

 

## <font color='red'>Assigment related to cost function</font>
1. Could you name any other cost function which you might use in regression problem?
2. Why Mean Squared error function is a good cost function

In [ ]:
# Try each integer for beta 0 and beta 1 some numbers
beta_0 = np.linspace(beta_0-3, beta_0+3, 50)
beta_1 = np.linspace(beta_1-5, beta_1+5, 50)


# All combinations of beta_0 and beta_1
plt_beta_0, plt_beta_1 = np.meshgrid(beta_0, beta_1)  # use need to understand this, plz play with this function

def calculate_mse(beta_0, beta_1):
    y_hat = beta_0 + beta_1 * x
    error = y_hat - y
    sse = np.sum(error ** 2)
    return ((1 / (2 * len(x))) * sse)

calculate_mse_v = np.vectorize(calculate_mse)
mse = calculate_mse_v(plt_beta_0, plt_beta_1)#.reshape(len(plt_beta_0), len(plt_beta_1))

fig = plt.figure(figsize=(10, 5))
ax = fig.gca(projection='3d')

surf = ax.plot_surface(plt_beta_0,
                       plt_beta_1,
                       mse,
                       cmap='viridis',
                       linewidth=0,
                       antialiased=False)
ax.set_xlabel(r'$\hat{\beta_0}$')
ax.set_ylabel(r'$\hat{\beta_1}$')
ax.set_zlabel(r'MSE')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()


Lets find the indexes of $\beta_{0}$ and $\beta_{1}$ where the cost function is minimum

In [ ]:
# Get the index for the values of beta_0 and beta_1
# For which SSE is lowest
beta_1_idx, beta_0_idx = np.unravel_index(mse.argmin(), mse.shape)


# Retrieve values of beta_0 and beta_1 for which
# SSE is lowest
beta_0_hat = beta_0[beta_0_idx]
beta_1_hat = beta_1[beta_1_idx]


# Print model parameters
print("Grid Search solution y = {} + {}x".format(beta_0_hat, beta_1_hat))


# Plot a line for our model
plt.scatter(x, y)
plt.plot(
    [min(x), max(x)],
    [min(x) * beta_1_hat + beta_0_hat, max(x) * beta_1_hat + beta_0_hat],
    color='blue'
)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Numerical solution to find minimum using scipy.optimize.minimum

In [ ]:
def linear_regression(beta, x ,y):
    model = beta[0] + beta[1]*x
    error = y - model
    return np.sum(error**2) / (2*len(x))
    
results = minimize(linear_regression, [0,4], args = (x,y), method='L-BFGS-B')


In [ ]:
print("Numerical solution using scipy,  y = {:.2f} + {:.2f}x".format(results.x[0], results.x[1]))


In [ ]:
plt.scatter(x, y, marker='s', color='green')
plt.plot(
    [min(x), max(x)],
    [min(x) * beta_1_hat + beta_0_hat, max(x) * beta_1_hat + beta_0_hat],
    color='blue', label='Grid Search method'
)

plt.plot(
    [min(x), max(x)],
    [min(x) * results.x[1] + results.x[0], max(x) * results.x[1] + results.x[0]],
    color='red', label='Numerical Solution'
)


plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

You might have notice that although it is easy to understand the grid search method to find the minimum in a cost fuction, but this is very inefficient, and quickly becomes slower as you increase granularity and search space. To overcome problem of inefficiney, we will try gradient descent

<a id='gradientdescent'></a>
# Gradient Descent

Gradient descent is an iterative process:

1. Initialise $\beta_{0}$ and $\beta_{1}$ with random values and calculate MSE
2. Calculate the gradient so we move in the direction of minimising MSE
3. Adjust the $\beta_{0}$ and $\beta_{1}$ with gradient
4. Use new weights to get values for $y_{model}$  to calculate MSE
5. Repeat steps 2-4

$$ \beta_{i}= \beta_{i} - \alpha\frac{\partial }{\partial \beta_{i}} MSE(\beta_{0},\beta_{1})$$
where "i" have two value 0 and 1. "$\alpha$" defines the learning rate (How fast you want to move down the slope)

$$\frac{\partial }{\partial \beta_{0}} MSE(\beta_{0},\beta_{1}) = \frac{\partial }{\partial \beta_{0}} \frac{1}{2n}\sum\limits_{i=1}^n (y_{i}^{actual} - (\beta_{0} + \beta_{1}x_{i}))^2 = - \frac{1}{n}\sum\limits_{i=1}^n (y_{i}^{actual} - (\beta_{0} + \beta_{1}x_{i}))$$


$$\frac{\partial }{\partial \beta_{1}} MSE(\beta_{0},\beta_{1}) = \frac{\partial }{\partial \beta_{1}} \frac{1}{2n}\sum\limits_{i=1}^n (y_{i}^{actual} - (\beta_{0} + \beta_{1}x_{i}))^2 = - \frac{1}{n}\sum\limits_{i=1}^n x_{i} (y_{i}^{actual} - (\beta_{0} + \beta_{1}x_{i}))$$


<img src="img/gradient_cost_function.png" width=600, align="center" />

In [ ]:
X = np.array([np.ones(len(x)), x])

In [ ]:
alpha = 0.00001
iterations = 150
beta_0_hat = 5
beta_1_hat = 1
mses = []
y_hat_list = []

for i in range(1, iterations+1):
    y_hat = beta_0_hat * X[0] + beta_1_hat * X[1]
    y_hat_list.append(y_hat)
    error = y_hat - y
    sse = np.sum(error ** 2)
    mse = ((1 / (2 * len(X.T))) * sse)
    mses.append(mse)
    
    gradient = np.dot(X, error) / len(X.T)
    beta_0_hat = beta_0_hat - (gradient[0] * alpha)
    beta_1_hat = beta_1_hat - (gradient[1] * alpha)
    
    if i % 10000 == 0:
        print("Iteration {}, MSE={}, β0={}, β1={}".format(
            i, round(mse, 3), round(beta_0_hat, 3), round(beta_1_hat, 3)))

In [ ]:
plt.scatter(X[1], y)
plt.plot(
    [min(X[1]), max(X[1])],
    [min(X[1]) * beta_1_hat + beta_0_hat, max(X[1]) * beta_1_hat + beta_0_hat],
    color='blue'
)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
plt.title('Cost Function J')
plt.xlabel('No. of iterations')
plt.ylabel('Cost')
plt.plot(mses[:8000])
plt.show()


## Create an animation

In [ ]:
from matplotlib import animation

fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(( min(x)-5, max(x)+5))
ax.set_ylim((int(min(y))-10, int(max(y))+10))
points = ax.scatter(x, y, color='red')
ax.set_xlabel('X')
ax.set_ylabel('Y')
line, = ax.plot([], [], lw=2)
epoch_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

In [ ]:
def init():
    line.set_data([], [])
    epoch_text.set_text('epoch = 0')
    return line, epoch_text
  
def animate(i):
    line.set_data(x, y_hat_list[i])
    epoch_text.set_text('epoch = {}'.format(i))
    return line, epoch_text
  
anim = animation.FuncAnimation(fig, animate, init_func=init, repeat=False,
                               frames=len(y_hat_list), interval=100, blit=True)

In [ ]:
#conda install -c conda-forge ffmpeg use this command if you are getting error to install ffmepg player
from IPython.display import HTML
plt.rcParams['animation.ffmpeg_path'] = '/opt/local/anaconda/anaconda/envs/course-ml/bin/ffmpeg' # For google colab
HTML(anim.to_html5_video())

<a id='multivariate'></a>
# Multivariate analysis

We’re going to create a linear regression that follows the following relation:

$$y=\beta_{0}+\beta_{1}x1 + \beta_{2}x2$$
where:
$$\beta_{0}=-10$$
$$\beta_{1}=2$$
$$\beta_{2}=3$$


Let’s start by creating some mock data.

In [ ]:
beta_0 = -10
beta_1 = 4
beta_2 = 2
x1= np.random.randint(10,100, 200)

x2= np.random.randint(-20,50, 200)

y = beta_0 + beta_1*x1 + beta_2*x2 

# Add some random error

y = y + np.random.normal(0,10, len(x))

In [ ]:
X = np.array([np.ones(len(x1)), x1,x2])
df = pd.DataFrame(X.T)
df.columns = ["x0", "x1", "x2"]
df.head()

In [ ]:
alpha = 0.00001
iterations = 200
beta_0_hat = -4
beta_1_hat = 1
beta_2_hat = 3
mses = []
y_hat_list = []

for i in range(1, iterations+1):
    y_hat = beta_0_hat * X[0] + beta_1_hat * X[1] + beta_2_hat * X[2]
    y_hat_list.append(y_hat)
    error = y_hat - y
    sse = np.sum(error ** 2)
    mse = ((1 / (2 * len(X.T))) * sse)
    mses.append(mse)
    
    gradient = np.dot(X, error) / len(X.T)
    beta_0_hat = beta_0_hat - (gradient[0] * alpha)
    beta_1_hat = beta_1_hat - (gradient[1] * alpha)
    beta_2_hat = beta_2_hat - (gradient[2] * alpha)
    
    if i % 50 == 0:
        print("Iteration {}, MSE={}, β0={}, β1={}, β2={} ".format(
            i, round(mse, 3), round(beta_0_hat, 3), round(beta_1_hat, 3), round(beta_2_hat, 3)))

In [ ]:
plt.title('Cost Function J')
plt.xlabel('No. of iterations')
plt.ylabel('Cost')
plt.plot(mses)
plt.show()
